<a href="https://colab.research.google.com/github/Pratiksha100/AnalyticsProject/blob/cheewan/pyconcorde_upgrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install git+https://github.com/jvkersch/pyconcorde

In [2]:
# install tweak file
%cp '/content/drive/MyDrive/Analytics Project/Concorde/concorde_class.py' '/content'
%cp '/content/drive/MyDrive/Analytics Project/Concorde/test.py' '/content'
%cp '/content/drive/MyDrive/Analytics Project/Concorde/concorde_to_replace/tsp.py' '/usr/local/lib/python3.10/dist-packages/concorde'
%cp '/content/drive/MyDrive/Analytics Project/Concorde/concorde_to_replace/util.py' '/usr/local/lib/python3.10/dist-packages/concorde'

In [ ]:
'''
%%capture
!cp "/content/drive/MyDrive/Analytics Project/Concorde.zip" "/content"
!unzip Concorde.zip
!bash /content/Concorde/install_concorde.sh
'''

In [1]:
import numpy as np
import pandas as pd
import math

In [ ]:
Warmsen_01_distance = pd.read_csv('/content/drive/MyDrive/Analytics Project/AOP_DP_Analytics/Data/Instances/Warmsen/distances/distances_31600-01.dat' ,
                                  names=['col1', 'col2', 'col3']
                                  )
# distance_index = Warmsen_01_distance['col1'].value_counts().index
check = Warmsen_01_distance.set_index(['col1', 'col2'])['col3'].unstack().values

In [2]:
import pandas as pd
import os
import numpy as np


def create_dist_mat_from_atsp(path_of_file): #input : path of the .atsp file
    # specify the path to the .atsp file
    file_path = path_of_file
    print(file_path)


    # initialize variables
    n_nodes = None
    dist_matrix = []

    # open and read the file
    with open(file_path) as f:
        for line in f:
            # extract the number of nodes
            if "DIMENSION" in line:
                n_nodes = int(line.split(":")[1].strip())
            # extract the distance matrix
            elif "EDGE_WEIGHT_SECTION" in line:
                while True:
                    line = f.readline()
                    if "EOF" in line:
                        break
                    dist_matrix.append(list(map(int, line.split())))
            # continue reading until end of file
            elif "EOF" in line:
                break


    #resizing the distance matrix to a square matrix
    list_of_lists = dist_matrix

    flat_list = []
    for sublist in list_of_lists:
        for item in sublist:
            flat_list.append(item)


    # Example 1D list
    lst = flat_list

    # Calculate the size of the square matrix
    n = len(lst)
    size = int(np.sqrt(n))

    # Reshape the 1D list into a 2D numpy array with the desired shape
    arr = np.reshape(lst, (size, size))

    # Print the resulting array
    dist_mat = np.array(arr)

    print("No of cities : "  , n_nodes)
    print(dist_mat.shape)
    return dist_mat #returns the distance matrix

In [7]:
%%time
from concorde_class import Concorde

distances = create_dist_mat_from_atsp('/content/drive/MyDrive/Analytics Project/TSPLIB_ATSP/rbg443.atsp')

concorde = Concorde()

concorde_sol = concorde.run_ProbATSP(distances)
print('-------------------------------------------------------------------------')
print(concorde.route)
print('-------------------------------------------------------------------------')
print(concorde.optimal_value)

/content/drive/MyDrive/Analytics Project/TSPLIB_ATSP/rbg443.atsp
No of cities :  443
(443, 443)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[112, 112, 115, 116, 123, 118, 123, 123, 123, 123, 114, 123, 123, 123, 110, 119, 111, 121, 118, 115, 119, 123, 121, 123, 114, 116, 120, 110, 112, 121, 123, 118, 112, 118, 115, 120, 123, 116, 123, 120, 110, 112, 121, 123, 113, 114, 121, 125, 116, 121, 115, 120, 121, 123, 114, 124, 116, 121, 112, 123, 114, 118, 114, 123, 113, 120, 123, 123, 123, 112, 123, 123, 123, 116, 118, 125, 112, 115, 116, 114, 118, 115, 106, 112, 124, 120, 118, 116, 112, 115, 111, 114, 123, 120, 121, 115, 121, 125, 110, 114, 109, 115, 121, 125, 118, 121, 110, 114, 121, 123, 121, 115, 123, 112, 123, 123, 115, 116, 115, 123, 123, 114, 112, 114, 116, 112, 115, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 116, 118, 115, 115, 115, 125, 125, 118, 121, 112, 123, 112, 113, 123, 123, 112, 112, 123, 123, 112, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 111, 113, 123, 110, 115, 123, 123, 112, 118, 123, 120, 123, 123, 116, 113, 110, 123, 111, 113, 123, 123, 123, 123, 121, 110, 123, 123, 123, 123, 123, 123, 112,